# 패키지 설치 및 드라이브 마운트

In [1]:
!pip install pillow==6.2.2
!pip install tqdm
!pip install terminaltables

패키지 설치 시 `RESTART RUNTIME` 버튼이 나온다면 한 번 RESTART 시켜주고 다시 실행한다.

In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [3]:
import os
os.chdir('/content/gdrive/My Drive/mask-detection-yolov3')
os.listdir('/content/gdrive/My Drive/mask-detection-yolov3')

['validate.py',
 '.gitignore',
 'image_detect.py',
 'models.py',
 'cam_detect.py',
 'requirements.txt',
 'train.py',
 'video_detect.py',
 'data',
 '.idea',
 'weights',
 'config',
 '.git',
 'testing',
 'utils',
 '__pycache__',
 'checkpoints',
 'mask yolov3 testing.ipynb',
 'mask yolov3 training.ipynb']

# Training

## Package import

In [4]:
from __future__ import division

from models import Darknet
from utils.utils import load_classes, weights_init_normal
from utils.datasets import ListDataset
from utils.parse_config import parse_data_config
from validate import evaluate

from terminaltables import AsciiTable

import os
import time
import argparse

import torch
from torch.utils.data import DataLoader
from torch.autograd import Variable

import warnings
import easydict
warnings.filterwarnings("ignore", category=UserWarning)

# from utils.logger import *

## Training Option Setting
Training Option을 정해준다. 필요한 몇 가지 옵션을 설명하면, epoch은 Training 데이터셋을 몇 번 반복해서 학습할 것인가를 말하며 batch_size는 한 번에 학습할 이미지의 갯수이다. pretrained_weights는 기학습된 모델의 weights 경로이며, checkpoint_interval은 몇 epoch 마다 학습한 모델을 저장할 것인지를 말하며, evaluation_interval은 몇 epoch 마다 모델을 validation set으로 평가할 것인지를 말한다.

이번 실습에서는 시간 상 30 epoch까지 학습한 모델을 40 epoch까지 학습하는 training을 진행하려고 한다.

In [6]:
opt = easydict.EasyDict({ 
    'epochs': 10, 
    'batch_size': 8, 
    'gradient_accumulations': 2, 
    'model_def': "config/yolov3_mask.cfg", 
    'data_config': "config/mask_dataset.data",
    'pretrained_weights': "checkpoints/yolov3_ckpt_30.pth", # weights/yolov3.weights
    "n_cpu": 8,
    'img_size': 416,
    'checkpoint_interval': 10,
    'evaluation_interval': 5,
    'compute_map': True,
    'multiscale_training': True
})
print(opt)
# logger = Logger("logs")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# gpu를 사용가능하면 device에 cuda를 저장한다. 

os.makedirs("checkpoints", exist_ok=True)
# checkpoints 폴더를 생성

{'epochs': 10, 'batch_size': 8, 'gradient_accumulations': 2, 'model_def': 'config/yolov3_mask.cfg', 'data_config': 'config/mask_dataset.data', 'pretrained_weights': 'checkpoints/yolov3_ckpt_30.pth', 'n_cpu': 8, 'img_size': 416, 'checkpoint_interval': 10, 'evaluation_interval': 5, 'compute_map': True, 'multiscale_training': True}


## 데이터와 모델 로딩

In [7]:
# 데이터셋의 경로와 클래스의 이름 등을 가져온다.
data_config = parse_data_config(opt.data_config)
train_path = data_config["train"]
valid_path = data_config["valid"]
class_names = load_classes(data_config["names"])

# dataset의 경로를 통해 ListDataset(dataset을 상속받은 class) 인스턴스를 만들고 이를 쉽게 가져올 수 있는 dataloader를 만든다.
dataset = ListDataset(train_path, augment=True, multiscale=opt.multiscale_training)
dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=opt.batch_size,
    shuffle=True,
    num_workers=opt.n_cpu,
    pin_memory=True,
    collate_fn=dataset.collate_fn,
)

# model을 정해준다. model 폴더에서 Darknet class 내에 yolov3 모델이 저장되어 있다. 모델에 관한 자세한 사항은 Darknet을 참고하자.
model = Darknet(opt.model_def).to(device)
model.apply(weights_init_normal)

# pretrained_weights를 load한다.
if opt.pretrained_weights:
    if opt.pretrained_weights.endswith(".pth"):
        model.load_state_dict(torch.load(opt.pretrained_weights))
    else:
        model.load_darknet_weights(opt.pretrained_weights)

## Training

In [ ]:
optimizer = torch.optim.Adam(model.parameters()) # Adam optimizer를 사용하며 model의 parameter를 인자로 넣어준다.

# to get mAP
to_get_mAP = None

for epoch in range(opt.epochs): # 미리 설정한 epoch 만큼 반복
    model.train()
    start_time = time.time()
    epoch += 31 # 31~40 epoch
    for batch_i, (_, imgs, targets) in enumerate(dataloader): # dataloader 에서 batch_size 만큼씩 학습.
        batches_done = len(dataloader) * epoch + batch_i

        imgs = Variable(imgs.to(device))
        targets = Variable(targets.to(device), requires_grad=False)

        loss, outputs = model(imgs, targets) # model에서 부터 loss 계산
        loss.backward() # loss에 대한 gradient 계산

        if batches_done % opt.gradient_accumulations:
            optimizer.step() # 학습 진행
            optimizer.zero_grad() # gradient를 0으로 만든다.


        log_str = "---- [Epoch %d/%d, Batch %d/%d] ----" % (epoch, opt.epochs + 30, batch_i, len(dataloader))
        log_str += f"Total loss {loss.item()}"
        print(log_str)

        model.seen += imgs.size(0)

    if epoch % opt.evaluation_interval == 0: # validation set에 대해 mAP 성능평가
        try:
            print("\n---- Evaluating Model ----")
            # Evaluate the model on the validation set
            precision, recall, AP, f1, ap_class = evaluate(
                model,
                path=valid_path,
                iou_thres=0.5,
                conf_thres=0.5,
                nms_thres=0.5,
                img_size=opt.img_size,
                batch_size=4,
            )
            evaluation_metrics = [
                ("val_precision", precision.mean()),
                ("val_recall", recall.mean()),
                ("val_mAP", AP.mean()),
                ("val_f1", f1.mean()),
            ]
            # logger.list_of_scalars_summary(evaluation_metrics, epoch)

            # Print class APs and mAP
            ap_table = [["Index", "Class name", "AP"]]
            for i, c in enumerate(ap_class):
                ap_table += [[c, class_names[c], "%.5f" % AP[i]]]
            print(AsciiTable(ap_table).table)
            print(f"---- mAP {AP.mean()}")
            to_get_mAP = AP.mean()
        except:
            to_get_mAP = 999999999999

    if epoch % opt.checkpoint_interval == 0: # checkpoint_interval 마다 model weight 저장.
        torch.save(model.state_dict(), "checkpoints/yolov3_ckpt_{0}.pth".format(epoch))

---- [Epoch 31/40, Batch 0/340] ----Total loss 1.0498112440109253
---- [Epoch 31/40, Batch 1/340] ----Total loss 0.9420675039291382
---- [Epoch 31/40, Batch 2/340] ----Total loss 3.4257636070251465
---- [Epoch 31/40, Batch 3/340] ----Total loss 2.3439600467681885
---- [Epoch 31/40, Batch 4/340] ----Total loss 3.008242130279541
---- [Epoch 31/40, Batch 5/340] ----Total loss 2.772014856338501
---- [Epoch 31/40, Batch 6/340] ----Total loss 5.922479629516602
---- [Epoch 31/40, Batch 7/340] ----Total loss 1.9322395324707031
---- [Epoch 31/40, Batch 8/340] ----Total loss 1.5293818712234497
---- [Epoch 31/40, Batch 9/340] ----Total loss 1.652246356010437
---- [Epoch 31/40, Batch 10/340] ----Total loss 1.0404229164123535
---- [Epoch 31/40, Batch 11/340] ----Total loss 1.7698338031768799
---- [Epoch 31/40, Batch 12/340] ----Total loss 2.033287525177002
---- [Epoch 31/40, Batch 13/340] ----Total loss 5.161789894104004
---- [Epoch 31/40, Batch 14/340] ----Total loss 4.551518440246582
---- [Epoch 